In [2]:
import numpy as np
import pandas as pd

In [4]:
study_df = pd.read_excel("/Users/verano13/Downloads/887667.xlsx")

In [6]:
study_df.columns

Index(['underlying_nid', 'nid', 'field_citation_value', 'location_name',
       'location_id', 'sex', 'year_start', 'year_end', 'age_start', 'age_end',
       'measure', 'mean', 'lower', 'upper', 'bundle_id', 'seq', 'origin_seq',
       'origin_id', 'is_outlier', 'year_id', 'age_group_id', 'val', 'variance',
       'gbd_component', 'sub_component', 'me_id', 'cause_id', 'rei_id',
       'note_year', 'release', 'age_type', 'underlying_field_citation_value'],
      dtype='object')